In [5]:
import cv2
import os

def extract_frames(video_path, frame_directory):
    os.makedirs(frame_directory, exist_ok=True)

    # Read the video
    cap = cv2.VideoCapture(video_path)
    print(f"Processing {video_path}... Total Frames: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}")

    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}.")
        return

    frame_count = 0
    while True:
        # Read a frame
        success, frame = cap.read()
        
        # If read was successful, save the frame
        if success:
            frame_filename = os.path.join(frame_directory, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
        else:
            # No more frames to read
            break

    # Release the video capture object
    cap.release()
    print(f"Frames extracted for {video_path}: {frame_count}")

# Adjust this path to where your videos are stored 你的local path可能和我的不一样，需要改
# video_folder_path = '/Users/lyricli/Documents/Visualization for Machine Learning/final project/Normal'
video_folder_path = '/Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash'


# Adjust this path to where you want to save the frames
# frames_save_path = 'data/frames/Normal'
frames_save_path = 'data/frames/Crash'

# Get a list of all video files
video_files = [f for f in os.listdir(video_folder_path) if f.endswith('.mp4')]

# Process each video file
for video_file in video_files:
    video_path = os.path.join(video_folder_path, video_file)
    video_name = video_file.split('.')[0]  # Assuming the file name is the video name
    frame_directory = os.path.join(frames_save_path, video_name)
    
    extract_frames(video_path, frame_directory)

Processing /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/000548.mp4... Total Frames: 50
Frames extracted for /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/000548.mp4: 50
Processing /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/001130.mp4... Total Frames: 50
Frames extracted for /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/001130.mp4: 50
Processing /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/001124.mp4... Total Frames: 50
Frames extracted for /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/001124.mp4: 50
Processing /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/000574.mp4... Total Frames: 50
Frames extracted for /Users/lyricli/Documents/Visualization for Machine Learning/final project/Crash/000574.mp4: 50
Processing /Users/lyricli/Documents/Visualizatio

In [11]:
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [17]:
# Custom dataset 还没写完！现在是用两个video做个试验
class DrivingFramesDataset(Dataset):
    def __init__(self, frame_dirs, labels, transform=None):
        """
        Args:
            frame_dirs (list): List of directories with frames.
            labels (list): List of labels for the directories.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.frame_dirs = frame_dirs
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.frame_dirs)

    def __getitem__(self, idx):
        frames = []
        for frame_filename in os.listdir(self.frame_dirs[idx]):
            img_path = os.path.join(self.frame_dirs[idx], frame_filename)
            image = Image.open(img_path)
            if self.transform:
                image = self.transform(image)
            frames.append(image)
        frames = torch.stack(frames)  # Convert list of frames to a tensor
        return frames, self.labels[idx]

# Define your transforms
transform = transforms.Compose([
    transforms.ToTensor(),  # This also scales pixel values to [0, 1]
])

# Create your dataset
dataset = DrivingFramesDataset(frame_dirs=['data/frames/Crash/000001', 'data/frames/Normal/000001'],
                              labels=[1, 0],  # 1 for crash, 0 for normal
                              transform=transform)

# Data loader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Now you can iterate over the data_loader during training
for frames, labels in data_loader:
    print(frames,labels)
    # Your training loop here
    pass


tensor([[[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000